### 1. sqlalchemy 란

- SQLAlchemy는 파이썬에서 사용할 수 있는 ORM(Object-Relational Mapping) 라이브러리로, 데이터베이스와의 상호작용을 추상화하여 데이터베이스를 쉽게 다룰 수 있게 해줍니다. ORM을 통해 데이터베이스 테이블을 파이썬 클래스로 매핑하고, 파이썬 코드를 사용하여 데이터를 쿼리하고 조작할 수 있습니다. 이를 통해 개발자는 SQL 쿼리를 직접 작성하는 대신 객체 지향적인 방식으로 데이터베이스를 다룰 수 있습니다.

- sqlalchemy 구성

![Alt text for broken image link](../resources/sqlalchemy.jpg)

- 데이터베이스와 상호 작용하는 방식은 추상화 수준에 따라  SQL 방식, Core 방식, ORM 방식 3가지로 나뉘어 집니다.

### 2. 데이터베이스 접속
- sqlalchemy 설치

In [ ]:
%pip install SQLAlchemy

- sqlite db 접속 engine 생성

In [ ]:
from sqlalchemy import create_engine

# sqlite:///mydb.db : dialect로 sqlite를 사용하고 db(파일)은 현재 directory의 mydb.db를 사용
# echo=True : 실행되는 SQL을 출력
engine = create_engine('sqlite:///mydb.db', echo=True)

### 3. SQL 방식
- SQLAlchemy의 가장 기본적인 사용 방식으로, 순수한 SQL 쿼리를 사용하여 데이터베이스와 상호 작용합니다.
- 사용자가 직접 SQL 쿼리를 작성하고 실행할 수 있습니다.
- 데이터베이스와 직접적으로 상호 작용하는 경우나 복잡한 쿼리를 사용해야 하는 경우에 유용합니다.


#### 3-1. 데이터베이스 connection 획득
- engine으로 부터 데이터베이스 connection을 바로 획득합니다.

In [ ]:
conn = engine.connect()

# db file 생성 확인 

#### 3-2. DDL 실행 - Create Table

In [ ]:
from sqlalchemy import text
conn.execute(text("CREATE TABLE TEST (name text PRIMARY KEY, age int, location text)")) 

#### 3-3. Insert 실행

In [ ]:
insert_statement = "INSERT INTO TEST (name, age, location) \
    VALUES (:name, :age, :location)"

test_data = [
    {"name": "Alice", "age": 25, "location":"New York"},
    {"name": "Bob", "age": 30, "location":"Los Angeles"},
    {"name": "Charlie", "age": 35, "location":"Chicago"},
]

conn.execute(text(insert_statement), test_data)
conn.commit()

#### 3-4. Select 실행

In [ ]:
select_statement = "SELECT x, y FROM TEST"
result = conn.execute(text(select_statement))
for row in result:
    print(row)

#### 3-5. DDL 실행 - Drop Table

In [ ]:
conn.execute(text("DROP TABLE TEST")) 

### 4. ORM 방식
- SQLAlchemy의 ORM(Object-Relational Mapping)은 파이썬 클래스와 데이터베이스 테이블을 매핑하여 객체지향 프로그래밍 스타일로 데이터베이스와 상호 작용하는 방법을 제공합니다.
- 데이터베이스 테이블을 파이썬 클래스로 나타내고, 이러한 클래스를 사용하여 데이터를 쿼리하고 조작할 수 있습니다.
- ORM은 데이터베이스와의 상호 작용을 추상화하여 데이터베이스와의 상호 작용을 보다 객체지향적으로 만들어줍니다.
- SQLAlchemy ORM을 사용하면 SQL 쿼리를 직접 작성할 필요 없이 파이썬 객체를 통해 데이터베이스를 조작할 수 있습니다.
#### 4-1. Model기본 Meta정보를 획득
- Dialect로부터 데이터베이스 engine(여기서는 sqlite)에 해당하는 Model(데이터베이스 Table에 매핑되는 Class 객체) 기본 Class를 획득합니다. 

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base() # Model 기본 Class 획득

#### 4-2. Model 정의
- Model 기본 Class를 상속받아 Model(Table Layout)을 정의합니다.

In [ ]:
"""
prompt :

sqlalchemy.orm.declarative_base 의 결과를 상속 받은 data model 두개를 아래 조건에 맞게 선언하는 code를 작성하라
---
1. 
class 이름 : User
table 이름 : user_account
칼럼들 :
    nickname (type:string)
    real_name  (type:string)
2. 
class 이름 : Asset
table 이름 : user_asset
칼럼들 :
    asset_name (type:string)
    owner_id  (type:integer, user_account.id를 참조하는 foreign key)

기타조건 : 
- user_account table 한 row 가 삭제되면 user_asset.owner_id로 연결된 row들도 삭제
- table 객체를 print하면 table의 column과 값을 출력한다.
===
results from ChatGPT-3.5
"""
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = 'user_account'  # 데이터베이스에서 사용할 테이블 이름입니다.

    id = Column(Integer, primary_key=True)
    nickname = Column(String(30))
    real_name = Column(String)

    # User와 Asset의 관계를 설정합니다.
    assets = relationship("Asset", cascade="all, delete-orphan")

    # 객체를 print 하면 호출됨
    def __repr__(self):
        return f"User(id={self.id}, nickname={self.nickname}, real_name={self.real_name})"
    
class Asset(Base):
    __tablename__ = 'user_asset'
    id = Column(Integer, primary_key=True)
    asset_name = Column(String(30))
    
    # User 테이블의 id를 외래키로 설정하여 User와의 관계를 맺습니다.
    owner_id = Column(Integer, ForeignKey('user_account.id'))

    # 객체를 print 하면 호출됨
    def __repr__(self):
        return f"Asset(id={self.id}, asset_name={self.asset_name}, owner_id={self.owner_id})"


- Model 기본 Class(Base)에 Model의 정의된 내용이 등록된 것을 확인합니다.

In [ ]:
print('tables : ', Base.metadata.tables)

#### 4-3. Table들 일괄 Create
- 데이터베이스에 Base 객체에 등록된 Model정보들을 Create Table 명령으로 전환하여 일괄 실행합니다.

In [ ]:
Base.metadata.create_all(engine)

- 전체 Table들 Drop 시에는 아래 code를 실행합니다.

In [ ]:
#Base.metadata.drop_all(engine)

#### 4-4. 데이터베이스 세션(session) 획득
- Session 객체를 통해 Connection Pooling으로부터 데이터베이스 session을 하나 획득합니다.

In [ ]:
from sqlalchemy.orm import Session

session = Session(engine)

#### 4-5. Insert 실행

In [ ]:
me = User(nickname='hennry', real_name='김형기')

session.add(me)

In [ ]:
session.commit()

In [ ]:
next_one = User(nickname='sunsiny', real_name='이순신')

session.add(next_one)

session.rollback()

#### 4-6. Select 실행

In [ ]:
users = session.query(User).all()

for user in users:
    print(user)

In [ ]:
user = session.query(User).first()

print(user)

In [ ]:
user = session.query(User).filter(User.nickname=='hennry')

for user in users:
    print(user)

#### 4-7. Update 실행

In [ ]:
user.real_name = '김형기2'
user.nickname = 'hennry2'
print('Before commit : Is the session dirty?',user in session.dirty)
session.commit()
print('After commit : Is the session dirty?',user in session.dirty)

#### 4-8. Delete 실행

In [ ]:
session.delete(user)
session.commit()

#### 4-9. Child Table Insert 실행

In [ ]:
me = User(nickname='tiffanie', real_name='김형기')
me.assets.append(Asset(asset_name="Car"))
me.assets.append(Asset(asset_name="House"))
session.add(me)
session.commit()

In [ ]:
me.assets.append(Asset(asset_name="Wife"))
me.assets.append(Asset(asset_name="Son1"))
me.assets.append(Asset(asset_name="Son2"))
session.commit()

In [ ]:
for asset in me.assets:
    print(asset)

#### 4-10. Child Table로부터 Select 실행

-    - Asset Table을 직접 Select

In [ ]:
assets = session.query(Asset).filter(Asset.owner_id==me.id, Asset.asset_name=='Car')

for asset in assets:
    print(asset)

-    - Table Join 방식으로 Select

In [ ]:
rows = session.query(User, Asset).join(Asset, Asset.owner_id==User.id).filter(User.id==me.id)

for row in rows:
    print(row[0], row[1])

-    - Comprehention을 사용해 me 객체에서 Select

In [ ]:
# asset_name으로 검색이 가능하도록 key=asset_name, value=Asset instance인 dictionary 생성
assets_in_me = { asset.asset_name:asset for asset in me.assets }

print('Assets : ', assets_in_me)
print('asset_name이 Car인 Asset  : ', assets_in_me['Car'])
print('asset_name이 Car인 Asset의 id  : ', assets_in_me['Car'].id)

In [ ]:
asset = next( a for a in me.assets if a.asset_name=="Car" )

print('asset_name이 Car인 Asset  : ', asset)
print('asset_name이 Car인 Asset의 id  : ', asset.id)

#### 4-11. Child Table Update 실행

In [ ]:
asset.asset_name = "비싼 Car"

In [ ]:
session.commit()

#### 4-12. Child Table Delete 실행

In [ ]:
me.assets.remove(asset)

In [ ]:
session.commit()

### 5. Core 방식
- SQLAlchemy Core는 SQL 표현식과 SQL 문을 생성하는 파이썬의 저수준 API입니다.
- 파이썬 코드를 사용하여 SQL 쿼리를 생성하고 실행할 수 있습니다.
- SQL 쿼리를 직접 작성하는 것보다는 추상화된 수준에서 데이터베이스와 상호 작용할 수 있습니다.

#### 5-1. Insert 실행

In [ ]:
from sqlalchemy import insert

stmt = insert(User).values(nickname='kang', real_name="강인모")

session.execute(stmt)
session.commit()

#### 5-2. Select 실행

In [ ]:
from sqlalchemy import select

stmt = select(User).where(User.nickname == 'kang')

cursor = session.execute(stmt)

for row in cursor:
    print(row)

#### 5-3. Update 실행

In [ ]:
from sqlalchemy import update

stmt = update(User).where(User.nickname == 'kang').\
    values(real_name='강감찬')

session.execute(stmt)
session.commit()

#### 5-4. Delete 실행

In [ ]:
from sqlalchemy import delete

stmt = delete(User).where(User.nickname == 'kang')

session.execute(stmt)
session.commit()

#### 5-5. Child Table Insert 실행

In [ ]:
# Insert User
user_insert = insert(User).values(nickname='mansour', real_name='만수르')
result = session.execute(user_insert)

user_id = result.lastrowid

# Insert Asset
asset1_insert = insert(Asset).values(asset_name="맨체스터 시티 FC", owner_id=user_id)
asset2_insert = insert(Asset).values(asset_name="돈", owner_id=user_id)

asset1_id = session.execute(asset1_insert).lastrowid
asset2_id = session.execute(asset2_insert).lastrowid

session.commit()

#### 5-6. Select join tables 실행

In [ ]:
stmt = select(User, Asset).where(User.id == user_id, User.id == Asset.owner_id)

rows = session.execute(stmt)

# 결과 출력
for row in rows:
    print("User ID:", row[0].id, "Nickname:", row[0].nickname, "Asset ID:", row[1].id, "Asset Name:", row[1].asset_name)